In [1]:
from  sklearn.datasets import load_digits

In [2]:
digits = load_digits()

In [3]:
features, targets = digits.data, digits.target
features.shape, targets.shape

((1797, 64), (1797,))

In [4]:
img = features[32].reshape(8,8).astype(int)
img

array([[ 0,  2, 13, 16, 16, 16, 11,  0],
       [ 0,  5, 16, 10,  5,  4,  1,  0],
       [ 0,  6, 16,  7,  3,  0,  0,  0],
       [ 0,  9, 16, 16, 16,  6,  0,  0],
       [ 0,  3,  8,  4, 11, 15,  0,  0],
       [ 0,  0,  0,  1, 12, 15,  0,  0],
       [ 0,  0,  4, 13, 16,  6,  0,  0],
       [ 0,  2, 16, 15,  8,  0,  0,  0]])

In [5]:
import pandas as pd
df = pd.DataFrame(img)
df

,0,1,2,3,4,5,6,7
0,0,2,13,16,16,16,11,0
1,0,5,16,10,5,4,1,0
2,0,6,16,7,3,0,0,0
3,0,9,16,16,16,6,0,0
4,0,3,8,4,11,15,0,0
5,0,0,0,1,12,15,0,0
6,0,0,4,13,16,6,0,0
7,0,2,16,15,8,0,0,0


In [6]:
df.style.set_properties(**{'font-size':'8pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7
0,0,2,13,16,16,16,11,0
1,0,5,16,10,5,4,1,0
2,0,6,16,7,3,0,0,0
3,0,9,16,16,16,6,0,0
4,0,3,8,4,11,15,0,0
5,0,0,0,1,12,15,0,0
6,0,0,4,13,16,6,0,0
7,0,2,16,15,8,0,0,0
